In [2]:
!pip install pyspark==3.1.2 py4j==0.10.9

     |████████████████████████████████| 212.4 MB 56 kB/s 
     |████████████████████████████████| 198 kB 49.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=56956cae342218b906df1f8bea79fba6060ce3a3efe9350f9479e8aa3c534235
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.stat import Correlation
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('spotify_classification')\
        .getOrCreate()

In [4]:
from google.colab import drive               
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = spark.read.json("drive/My Drive/dataset.json")

In [6]:
df.show(5)

+------------+------------------+--------------------+---------------------+------------+-----------+------+--------------------+--------------------+----------------+---+--------+--------+----+----------------+------------+-----------+--------------------+---------------------+-------+--------------+-------+
|acousticness|               age|avg_artist_followers|avg_artist_popularity|danceability|duration_ms|energy|              genres|            id_track|instrumentalness|key|liveness|loudness|mode|popularity_track|release_date|speechiness|sum_artist_followers|sum_artist_popularity|  tempo|time_signature|valence|
+------------+------------------+--------------------+---------------------+------------+-----------+------+--------------------+--------------------+----------------+---+--------+--------+----+----------------+------------+-----------+--------------------+---------------------+-------+--------------+-------+
|       0.658| 41.83013698630137|              5403.5|             

In [7]:
from pyspark.ml.feature import QuantileDiscretizer 

qds = QuantileDiscretizer(relativeError=0.0001, handleInvalid="error", numBuckets=10, inputCol="popularity_track", outputCol="label")

df = qds.setHandleInvalid("keep").fit(df).transform(df)

In [8]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler( 
inputCols=[
  'age',
 'duration_ms',
 'danceability',
 'energy',
 'key',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature',
 'sum_artist_followers',
 'sum_artist_popularity',
 'avg_artist_popularity'], 
outputCol="feat")
df=assembler.transform(df)

In [9]:
from pyspark.ml.feature import StandardScaler


scaler = StandardScaler(inputCol="feat", outputCol="scaledFeatures")


scalerModel = scaler.fit(df)


df = scalerModel.transform(df)

In [11]:
from pyspark.ml.feature import VarianceThresholdSelector

selector = VarianceThresholdSelector(varianceThreshold=1,featuresCol='scaledFeatures', outputCol="features")

df = selector.fit(df).transform(df)

print("Output: Features with variance lower than %f are removed." %
      selector.getVarianceThreshold())

Output: Features with variance lower than 1.000000 are removed.


In [12]:
df.select("features").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[1.2361331222982912,2.1964380167735986,2.2425341538161,1.8900121366507203,0.0,2.5283072549510672,6.364381858546468,0.002309747438951722,1.9235483655073946]                    |
|[1.743569701544143,3.545337450987819,0.9547422635058642,1.5596908665673879,0.002985334272949299,3.492953715301628,8.485842478061958,0.004238847132111549,2.067814492920449]    |
|[1.9877358337892006,3.923347616918058,0.7771157958768663,1.3787322577391272E-4,3.442885505738011,0.8985215013

In [ ]:
final_data = df.select("id_track", "features","label")

In [ ]:
train, test = final_data.randomSplit([0.7, 0.3], seed = 10)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Naive Bayes

In [ ]:
from pyspark.ml.classification import NaiveBayes

In [ ]:
nb = NaiveBayes()                         
nbModel = nb.fit(train)
predictions = nbModel.transform(test)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

0.21566325483848908

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="f1")
evaluator.evaluate(predictions)

0.20108403172295577

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="recallByLabel")
evaluator.evaluate(predictions)

0.5882908962013351

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="precisionByLabel")
evaluator.evaluate(predictions)

0.40849981539976205

# LSVM one vs others

In [ ]:
from pyspark.ml.classification import LinearSVC, OneVsRest

In [ ]:
lsvc = LinearSVC(maxIter=10, regParam=0.1)

In [ ]:
ovr = OneVsRest(classifier=lsvc)

In [ ]:
ovrModel = ovr.fit(train)

In [ ]:
predictions = ovrModel.transform(test)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

0.23006474114938336

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="f1")
evaluator.evaluate(predictions)

0.15441934541802574

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="recallByLabel") 
evaluator.evaluate(predictions)

0.8157972470018314

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="precisionByLabel") 
evaluator.evaluate(predictions)

0.3300587982217123

# Logistic regression one vs others

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True)

In [ ]:
ovr = OneVsRest(classifier=lr)

In [ ]:
ovrModel = ovr.fit(train)

In [ ]:
predictions = ovrModel.transform(test)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

0.24001527344017143

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="f1")
evaluator.evaluate(predictions)

0.19257092638095072

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="recallByLabel") 
evaluator.evaluate(predictions)

0.7359839309978141

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="precisionByLabel") 
evaluator.evaluate(predictions)

0.404375486886523